# Data classification based on categories and sub categories

In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame
import nltk, string
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline

In [2]:
df = pd.read_excel("Data.xlsx")

In [3]:
dfclassification = pd.read_excel("Classification details.xlsx")

In [4]:
dfclassification

,Unnamed: 0,Code secteur,Nom Secteur
0,0,1.0,1 PRESTATIONS TRANSPORT & LOGISTIQUE
1,1,1.1,1.1 SERVICES TRANSPORT
2,2,NaN,1.1.1 TRANSPORTS ROUTIERS GÉNÉRAL CARGO
3,3,NaN,1111 - Transport routier régional et national ...
4,4,NaN,1112 - Transport de vrac solide
...,...,...,...
169,169,NaN,3224 - Location de matériel de stockage
170,170,NaN,3.2.4 EMBALLAGE
171,171,NaN,3241 - Fabricant de matériel d’emballage
172,172,NaN,"3242 - Calages, films, feuillards"


In [5]:
dfclassification = dfclassification.loc[:, ~dfclassification.columns.str.contains('^Unnamed')]

In [6]:
writer = pd.ExcelWriter('Classification details.xlsx')
dfclassification.to_excel(writer)
writer.save()

# Preparing sub categories

In [7]:
dfSubCategories = DataFrame(columns=['Code Secteur','Nom Secteur'])

In [8]:
Code_Secteur = []
Secteur = []

for i in range(174):
    if dfclassification['Nom Secteur'][i][0:5].find('.') == -1:
        Code_Secteur.append(dfclassification['Nom Secteur'][i][0:5])
        Secteur.append(dfclassification['Nom Secteur'][i][5:])

In [9]:
len(Secteur)

126

In [10]:
len(Code_Secteur)

126

In [11]:
dfSubCategories['Code Secteur'] = Code_Secteur
dfSubCategories['Nom Secteur'] = Secteur

In [12]:
dfSubCategories.head()

,Code Secteur,Nom Secteur
0,1 P,RESTATIONS TRANSPORT & LOGISTIQUE
1,1111,- Transport routier régional et national par lots
2,1112,- Transport de vrac solide
3,1113,- Groupage et distribution
4,1122,- Transport de meubles


In [13]:
dfSubCategories = dfSubCategories.iloc[1:]

In [14]:
dfSubCategories.head()

,Code Secteur,Nom Secteur
1,1111,- Transport routier régional et national par lots
2,1112,- Transport de vrac solide
3,1113,- Groupage et distribution
4,1122,- Transport de meubles
5,1123,- Transport de produits d’hygiène-santé


In [15]:
new_index = np.arange(125)

In [16]:
dfSubCategories = dfSubCategories.reindex(new_index, fill_value=' ')

In [17]:
dfSubCategories.head()

,Code Secteur,Nom Secteur
0,,
1,1111,- Transport routier régional et national par lots
2,1112,- Transport de vrac solide
3,1113,- Groupage et distribution
4,1122,- Transport de meubles


# Preparing categories

In [18]:
dfCategories = DataFrame(columns=['Code Secteur','Nom Secteur'])

In [19]:
Code_Secteur = []
Secteur = []

for i in range(174):
    if dfclassification['Nom Secteur'][i][0:5].find('.') != -1:
        if dfclassification['Nom Secteur'][i][0:5][3] == '.':
            Code_Secteur.append(dfclassification['Nom Secteur'][i][0:5])
            Secteur.append(dfclassification['Nom Secteur'][i][5:])

In [20]:
dfCategories['Code Secteur'] = Code_Secteur
dfCategories['Nom Secteur'] = Secteur

In [37]:
dfCategories.head()

,Code Secteur,Nom Secteur
0,1.1.1,TRANSPORTS ROUTIERS GÉNÉRAL CARGO
1,1.1.2,TRANSPORTS SPÉCIALISÉS
2,1.1.3,TRANSPORTS EN CITERNE
3,1.1.4,MESSAGERIE
4,1.1.5,TRANSPORT EXPRESS OU URGENT


In [38]:
len(dfCategories['Nom Secteur'])

37

# Level 1 : sub categories

In [32]:
dfsub = df.copy()

In [33]:
dfsub = dfsub.loc[:, ~dfsub.columns.str.contains('^Unnamed')]

In [61]:
listData = []
for i in range(1589):
    listData.append(''.join(str([dfsub['Nom'][i],dfsub['Description'][i],dfsub['Secteur'][i],dfsub['Activite'][i]])))

['[\'Allo Fret\', " stockage , transport frigorifique , entreposage , transport de marchandises , entreposage d\'objet d\'art , transport de bagages , stockage de marchandise , transport national , logistique évènementielle , location d\'entrepôt , transport d\'oeuvre d\'art , transport de produits frais , stockage de meubles , commissionnaire de transport , expédition d\'oeuvres d\'art , groupage , transport de produits médicaux , réception de marchandises , expédition , transport européen , transfert industriel , préparation commande , transport de produits surgelés , gestion de commande", \'Transport : logistique,\\xa0Transport routier\', \'Transport : logistique     Transport routier      Transport express      Garde-meubles      Déménagements industriels, de bureaux      Entrepôts      Services, conseils en logistique      Transport ferroviaire      Transport international      Coursiers\']',
 '[\'DHL Express\', " Envois Express de documents et marchandises en France et à l\'inter

In [29]:
stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

'''remove punctuation, lowercase, stem'''
def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))

vectorizer = TfidfVectorizer(tokenizer=normalize, stop_words='english')

def cosine_sim(text1, text2):
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]

In [35]:
dfSubCatClass = DataFrame(columns=['Nom','Code Secteur','Nom Secteur'])
listNom = []
listCode = []
listNomSecteur = []
k=0
for item in listData:
    listNom.append(dfsub['Nom'][k])
    dictData = {}
    print("looking for : "+ dfsub['Nom'][k]+" index : "+str(k) )
    for j in range(125):
        dictData[dfSubCategories['Nom Secteur'][j]] = cosine_sim(item,dfSubCategories['Nom Secteur'][j])
        
    listCode.append( max(dictData.values()) )
    listNomSecteur.append( list(dictData.keys())[list(dictData.values()).index(max(dictData.values()))] )
    
    
    k = k+1
    print("\n")

looking for : Allo Fret index : 0


looking for : DHL Express index : 1


looking for : AAC Globe Express index : 2


looking for : France Express index : 3


looking for : DHL Express index : 4


looking for : DHL Express index : 5


looking for : Chimwa transit index : 6


looking for : GEODIS | Distribution & Express - Agence de Paris index : 7


looking for : Babel Taxi index : 8


looking for : DHL Express index : 9


looking for : City Cab Transfer index : 10


looking for : Chrono Drive Services index : 11


looking for : TAXI PREMIUM index : 12


looking for : Sotracars France index : 13


looking for : FRAISE index : 14


looking for : Sotracars France index : 15


looking for : City Cab Transfer index : 16


looking for : Chimwa transit index : 17


looking for : Babel Taxi index : 18


looking for : Chrono Drive Services index : 19


looking for : AZR Transports index : 20


looking for : Air Caraïbes index : 21


looking for : Air France index : 22


looking for : Air Franc



looking for : France Taxi index : 178


looking for : Aigle Azur index : 179


looking for : Ratp index : 180


looking for : Ratp REGIE AUTONOME TRANSPORTS PARISIENS index : 181


looking for : RATP - PHT NEY index : 182


looking for : Taxis (Appel Taxi) index : 183


looking for : Boutique SNCF Gaité index : 184


looking for : Excellence Plus index : 185


looking for : Roy Pascal index : 186


looking for : Sncf Mobilites index : 187


looking for : Allouache Djemel index : 188


looking for : El Ouadrassi Hatime index : 189


looking for : Bornes de Taxis index : 190


looking for : Ville De Paris index : 191


looking for : Nafii Nagat index : 192


looking for : Elbaz David index : 193


looking for : Joaquim Alvaro index : 194


looking for : L.R.R. (SARL) index : 195


looking for : Taxis Appel Taxi index : 196


looking for : BELKACEM KACI index : 197


looking for : Ville De Paris index : 198


looking for : Taxis Appel Taxi index : 199


looking for : TAXICOP index : 200



looking for : Asti index : 359


looking for : Francisco Roger index : 360


looking for : Airsource Partners index : 361


looking for : NMJ Transport index : 362


looking for : Transmarine index : 363


looking for : Ecolotrans index : 364


looking for : Voyage Europa Bus index : 365


looking for : Trans Tm index : 366


looking for : Travel Car France index : 367


looking for : 2 Sdti index : 368


looking for : Station de métro Opéra index : 369


looking for : Gsek Transports index : 370


looking for : Hamlet Voyages index : 371


looking for : Parisitour index : 372


looking for : Paris Vélopostale index : 373


looking for : Anais Course index : 374


looking for : Paris Airport Shuttle index : 375


looking for : Coursiers Parisiens index : 376


looking for : Transports Simba index : 377


looking for : AB Plus index : 378


looking for : Allocab index : 379


looking for : Ecotransite index : 380


looking for : Paris Best Way index : 381


looking for : Transp Logist



looking for : Izitrans index : 540


looking for : Chateau Alexandre index : 541


looking for : Rabahi Soraya index : 542


looking for : Tumu (SARL) index : 543


looking for : Boveire Sabri index : 544


looking for : Lanani Malika index : 545


looking for : Nundoo Sangreeta index : 546


looking for : Ladoui Aldjia index : 547


looking for : Private Driver Paris index : 548


looking for : Sides Jacques index : 549


looking for : Lupoli Enzo index : 550


looking for : Daoud Mustapha index : 551


looking for : Simao Grâce index : 552


looking for : De Abreu Tiago index : 553


looking for : Dabi Rudy index : 554


looking for : Paris Day Trip Private Tours index : 555


looking for : Dembele Salle index : 556


looking for : Tony Bruno index : 557


looking for : Boutin Olivier index : 558


looking for : Saadi Ladjel index : 559


looking for : Akli Malika index : 560


looking for : Coursier.fr index : 561


looking for : BEN MOUSSA ABDERAHMAN index : 562


looking for : A



looking for : El Mahjoubi Rabie index : 723


looking for : Trans'excel index : 724


looking for : Trans Auto 69 index : 725


looking for : Nord Sud Express 69 index : 726


looking for : Guidoum Mohammed index : 727


looking for : Atlas Drive index : 728


looking for : Dz Voyages index : 729


looking for : Lyon Univers index : 730


looking for : Rhone distribution index : 731


looking for : My express driver index : 732


looking for : Vélimotion index : 733


looking for : Publitour index : 734


looking for : SPALVIERI CORENTIN index : 735


looking for : Teffahi Courses index : 736


looking for : International Freight Logistics index : 737


looking for : Vélimotion index : 738


looking for : Rh trans logistics index : 739


looking for : Jhon Philippe index : 740


looking for : SOCIETE BECYCLE index : 741


looking for : 3t index : 742


looking for : Lp Express SARL index : 743


looking for : Chetioui Hugo index : 744


looking for : Madani Mohamed index : 745


look



looking for : FLY TRANS index : 910


looking for : At.At-Transports index : 911


looking for : Trans'Chasse index : 912


looking for : Chapelle Thierry index : 913


looking for : JetAirFly index : 914


looking for : TRANSNATURE index : 915


looking for : Murard Marc index : 916


looking for : ACTIV COLIS index : 917


looking for : Jarosson Thomas index : 918


looking for : Ndjibu Trésor index : 919


looking for : Diaby Nafie index : 920


looking for : Pichon Michel index : 921


looking for : Amolomalo Nathan Magnus index : 922


looking for : Amolomalo Nathan Magnus index : 923


looking for : Mekdad Abdelhamid index : 924


looking for : Clement Gilles index : 925


looking for : Berkani Karim index : 926


looking for : Fast Pack index : 927


looking for : BH TRANSERVICES index : 928


looking for : Samate Sadio index : 929


looking for : Davoine Marcel index : 930


looking for : Sandi Laiddine index : 931


looking for : URBAN LOGISTICS SERVICES index : 932


lookin



looking for : Shedy Trans index : 1092


looking for : Bettar Abdelkarim index : 1093


looking for : Trans Colis Maroc index : 1094


looking for : Rahali Lamine index : 1095


looking for : Taxiway index : 1096


looking for : Hadjadj index : 1097


looking for : De Villard Régis index : 1098


looking for : De Villard Régis index : 1099


looking for : Taxi Daniel index : 1100


looking for : TRANSPORT EXPRESS LYONNAIS index : 1101


looking for : Alphadis index : 1102


looking for : Bousdira Adnane index : 1103


looking for : Autocar-Location.com index : 1104


looking for : Trans World Logistics index : 1105


looking for : Alameddine Véronique index : 1106


looking for : Philips Jean-Marc index : 1107


looking for : Siergie Romain index : 1108


looking for : Couquet Christophe index : 1109


looking for : Bj Conseil index : 1110


looking for : Damnati Adib Rédouane index : 1111


looking for : Mc Catty Christina index : 1112


looking for : MKSB EXPRESS index : 1113


loo



looking for :  Transglory  index : 1265


looking for :  Transit Services Transport (Tst)  index : 1266


looking for :  Transport Rapide Oriental (T.r.o.)  index : 1267


looking for :  1er Choix Distribution  index : 1268


looking for :  2r Transports  index : 1269


looking for :  2sid  Tech  index : 1270


looking for :  2w Logistics  index : 1271


looking for :  3D Trans  index : 1272


looking for :  3N Tours  index : 1273


looking for :  3S Transit  index : 1274


looking for :  4 L Logistique Et Transport  index : 1275


looking for :  4 Line  index : 1276


looking for :  4 Rent - Transport  index : 1277


looking for :  4tl Med Transit  index : 1278


looking for :  4X4 Evasion  index : 1279


looking for :  A Five International Transport (A Five)  index : 1280


looking for :  A Plus Transport  index : 1281


looking for :  A Right Trans  index : 1282


looking for :  A Toute Vitesse Courses (Atvc)  index : 1283


looking for :  A&J Transport et Logistique  index : 1284



looking for :  Al Transports  index : 1428


looking for :  AlAdarissa Tours  index : 1429


looking for :  Alae Transport Et Materiaux  index : 1430


looking for :  Alam Transit Ammor (A.t.a.)  index : 1431


looking for :  Alaoui Transport  index : 1432


looking for :  Albatross Tours  index : 1433


looking for :  Alcus transport  index : 1434


looking for :  Alfa Evasion  index : 1435


looking for :  Alfa Tir  index : 1436


looking for :  AlHamd Logistique  index : 1437


looking for :  Alif Trans International  index : 1438


looking for :  Alina Transport  index : 1439


looking for :  Alintissar AlMouttahida  index : 1440


looking for :  Alisatransit  index : 1441


looking for :  Alitalia - Compagnia Aerea Italiana (Alitalia)  index : 1442


looking for :  Alitalia -Compagnia Aerea Italiana (Alitalia)  index : 1443


looking for :  Alitihad Négoce & Transport  index : 1444


looking for :  Alliance Logistics  index : 1445


looking for :  Alliance Shipping  index : 1446

In [38]:
len(listNom)

1589

In [36]:
len(listCode)

1589

In [37]:
len(listNomSecteur)

1589

In [39]:
dfSubCatClass['Nom'] = listNom
dfSubCatClass['Code Secteur'] = listCode
dfSubCatClass['Nom Secteur'] = listNomSecteur

In [45]:
dfSubCatClass

,Nom,Code Secteur,Nom Secteur
,,,
0,Allo Fret,0.563834,- Transport de meubles
1,DHL Express,0.235836,- Transport et livraison
2,AAC Globe Express,0.618945,- Transport de meubles
3,France Express,0.499754,- Transport express national
4,DHL Express,0.283636,- Transport et livraison
...,...,...,...
1584,Awell Ttl,0.161714,- Logistique de retour
1585,Axalys Logistics Maroc,0.150557,- Transport de meubles
1586,Aya Intercontinental,0.161714,- Transport de meubles


In [49]:
# dfSubCategories

dfSubCatClass.rename(columns = {"Code Secteur" : "Similarity"}, inplace = True)

In [55]:
dfSubCatClass

,Nom,Similarity,Nom Secteur
,,,
0,Allo Fret,0.563834,- Transport de meubles
1,DHL Express,0.235836,- Transport et livraison
2,AAC Globe Express,0.618945,- Transport de meubles
3,France Express,0.499754,- Transport express national
4,DHL Express,0.283636,- Transport et livraison
...,...,...,...
1584,Awell Ttl,0.161714,- Logistique de retour
1585,Axalys Logistics Maroc,0.150557,- Transport de meubles
1586,Aya Intercontinental,0.161714,- Transport de meubles


In [24]:
dfSubCatClass['Nom Secteur']

0             - Transport de meubles
1           - Transport et livraison
2             - Transport de meubles
3       - Transport express national
4           - Transport et livraison
                    ...             
1584          - Logistique de retour
1585          - Transport de meubles
1586          - Transport de meubles
1587          - Logistique de retour
1588          - Logistique de retour
Name: Nom Secteur, Length: 1589, dtype: object

In [26]:
listCode = []
listSecteur = []
for i in range(1589):
    for j in range(125):
        if  dfSubCatClass['Nom Secteur'][i] == dfSubCategories['Nom Secteur'][j]:
            listCode.append(dfSubCategories['Code Secteur'][j])
            listSecteur.append(dfSubCategories['Nom Secteur'][j])
            break

In [78]:
len(listCode)

1589

In [28]:
listSecteur

['- Transport de meubles',
 '- Transport et livraison',
 '- Transport de meubles',
 '- Transport express national',
 '- Transport et livraison',
 '- Transport et livraison',
 '- Transport express international',
 '- Transport et livraison',
 '- Transport urgent, transport à la demande',
 '- Transport et livraison',
 ' ',
 '- Transport urgent, transport à la demande',
 '- Transport urgent, transport à la demande',
 '- Transport en citerne alimentaire',
 '- Transport express national',
 '- Transport en citerne alimentaire',
 ' ',
 '- Transport express international',
 '- Transport urgent, transport à la demande',
 '- Transport urgent, transport à la demande',
 '- Transport de meubles',
 '- Transport de meubles',
 '- Transport de meubles',
 '- Transport de meubles',
 '- Transport de meubles',
 '- Transport en citerne alimentaire',
 '- Transport de meubles',
 '- Transport en citerne alimentaire',
 '- Transport de meubles',
 '- Transport de meubles',
 '- Transport de meubles',
 '- Transport

In [79]:
listCode

['1122 ',
 '1231 ',
 '1122 ',
 '1151 ',
 '1231 ',
 '1231 ',
 '1152 ',
 '1231 ',
 '1153 ',
 '1231 ',
 ' ',
 '1153 ',
 '1153 ',
 '1132 ',
 '1151 ',
 '1132 ',
 ' ',
 '1152 ',
 '1153 ',
 '1153 ',
 '1122 ',
 '1122 ',
 '1122 ',
 '1122 ',
 '1122 ',
 '1132 ',
 '1122 ',
 '1132 ',
 '1122 ',
 '1122 ',
 '1122 ',
 '1122 ',
 '1122 ',
 '1132 ',
 '1122 ',
 '1122 ',
 '1122 ',
 '1122 ',
 '1152 ',
 '1132 ',
 '1132 ',
 '1231 ',
 '1122 ',
 '1132 ',
 '1151 ',
 '1152 ',
 '1231 ',
 '1153 ',
 '1122 ',
 '1122 ',
 '1152 ',
 '1122 ',
 '1122 ',
 '1122 ',
 '1122 ',
 '1122 ',
 '1122 ',
 '1122 ',
 '1122 ',
 '1153 ',
 '1132 ',
 '1122 ',
 '1551 ',
 '1122 ',
 '1122 ',
 '1132 ',
 '1122 ',
 '1122 ',
 '1132 ',
 '1122 ',
 '1122 ',
 '1122 ',
 '1132 ',
 '1122 ',
 '1151 ',
 ' ',
 '1152 ',
 '1122 ',
 '1122 ',
 '1152 ',
 '1151 ',
 '1122 ',
 '1123 ',
 '1122 ',
 '1152 ',
 '1521 ',
 '1152 ',
 '1183 ',
 '1122 ',
 '1122 ',
 '1122 ',
 '1122 ',
 '1122 ',
 '1122 ',
 '1132 ',
 ' ',
 '1122 ',
 '1122 ',
 '1122 ',
 '1122 ',
 '1122 ',
 '1122

In [80]:
dfSubCatClass['Code Secteur'] = listCode

In [81]:
dfSubCatClass

,Nom,Similarity,Nom Secteur,Code Secteur
,,,,
0,Allo Fret,0.563834,- Transport de meubles,1122
1,DHL Express,0.235836,- Transport et livraison,1231
2,AAC Globe Express,0.618945,- Transport de meubles,1122
3,France Express,0.499754,- Transport express national,1151
4,DHL Express,0.283636,- Transport et livraison,1231
...,...,...,...,...
1584,Awell Ttl,0.161714,- Logistique de retour,1234
1585,Axalys Logistics Maroc,0.150557,- Transport de meubles,1122
1586,Aya Intercontinental,0.161714,- Transport de meubles,1122


In [82]:
writer = pd.ExcelWriter('Classification sub categories.xlsx')
dfSubCatClass.to_excel(writer)
writer.save()

In [22]:
dfSubCatClass = pd.read_excel("Classification sub categories.xlsx")

In [23]:
dfSubCatClass

,Unnamed: 0,Nom,Similarity,Nom Secteur,Code Secteur
0,0,Allo Fret,0.563834,- Transport de meubles,1122
1,1,DHL Express,0.235836,- Transport et livraison,1231
2,2,AAC Globe Express,0.618945,- Transport de meubles,1122
3,3,France Express,0.499754,- Transport express national,1151
4,4,DHL Express,0.283636,- Transport et livraison,1231
...,...,...,...,...,...
1584,1584,Awell Ttl,0.161714,- Logistique de retour,1234
1585,1585,Axalys Logistics Maroc,0.150557,- Transport de meubles,1122
1586,1586,Aya Intercontinental,0.161714,- Transport de meubles,1122
1587,1587,Ayan Transit,0.161714,- Logistique de retour,1234


# Level 2 : categories

In [39]:
dfCatClass = DataFrame(columns=['Nom','Code Secteur','Nom Secteur'])
listNom = []
listCode = []
listNomSecteur = []
k=0
for item in listData:
    listNom.append(dfsub['Nom'][k])
    dictData = {}
    print("looking for : "+ dfsub['Nom'][k]+" index : "+str(k) )
    for j in range(37):
        dictData[dfCategories['Nom Secteur'][j]] = cosine_sim(item,dfCategories['Nom Secteur'][j])
        
    listCode.append( max(dictData.values()) )
    listNomSecteur.append( list(dictData.keys())[list(dictData.values()).index(max(dictData.values()))] )
    
    
    k = k+1
    print("\n")

looking for : Allo Fret index : 0


C:\Users\USER\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'ha', 'henc', 'hereaft', 'herebi', 'hi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'thi', 'thu', 'togeth', 'twelv', 'twenti', 'veri', 'wa', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not in stop_words.
  warnin



looking for : DHL Express index : 1


looking for : AAC Globe Express index : 2


looking for : France Express index : 3


looking for : DHL Express index : 4


looking for : DHL Express index : 5


looking for : Chimwa transit index : 6


looking for : GEODIS | Distribution & Express - Agence de Paris index : 7


looking for : Babel Taxi index : 8


looking for : DHL Express index : 9


looking for : City Cab Transfer index : 10


looking for : Chrono Drive Services index : 11


looking for : TAXI PREMIUM index : 12


looking for : Sotracars France index : 13


looking for : FRAISE index : 14


looking for : Sotracars France index : 15


looking for : City Cab Transfer index : 16


looking for : Chimwa transit index : 17


looking for : Babel Taxi index : 18


looking for : Chrono Drive Services index : 19


looking for : AZR Transports index : 20


looking for : Air Caraïbes index : 21


looking for : Air France index : 22


looking for : Air France index : 23


looking for : Tunis


looking for : France Taxi index : 178


looking for : Aigle Azur index : 179


looking for : Ratp index : 180


looking for : Ratp REGIE AUTONOME TRANSPORTS PARISIENS index : 181


looking for : RATP - PHT NEY index : 182


looking for : Taxis (Appel Taxi) index : 183


looking for : Boutique SNCF Gaité index : 184


looking for : Excellence Plus index : 185


looking for : Roy Pascal index : 186


looking for : Sncf Mobilites index : 187


looking for : Allouache Djemel index : 188


looking for : El Ouadrassi Hatime index : 189


looking for : Bornes de Taxis index : 190


looking for : Ville De Paris index : 191


looking for : Nafii Nagat index : 192


looking for : Elbaz David index : 193


looking for : Joaquim Alvaro index : 194


looking for : L.R.R. (SARL) index : 195


looking for : Taxis Appel Taxi index : 196


looking for : BELKACEM KACI index : 197


looking for : Ville De Paris index : 198


looking for : Taxis Appel Taxi index : 199


looking for : TAXICOP index : 200




looking for : Francisco Roger index : 360


looking for : Airsource Partners index : 361


looking for : NMJ Transport index : 362


looking for : Transmarine index : 363


looking for : Ecolotrans index : 364


looking for : Voyage Europa Bus index : 365


looking for : Trans Tm index : 366


looking for : Travel Car France index : 367


looking for : 2 Sdti index : 368


looking for : Station de métro Opéra index : 369


looking for : Gsek Transports index : 370


looking for : Hamlet Voyages index : 371


looking for : Parisitour index : 372


looking for : Paris Vélopostale index : 373


looking for : Anais Course index : 374


looking for : Paris Airport Shuttle index : 375


looking for : Coursiers Parisiens index : 376


looking for : Transports Simba index : 377


looking for : AB Plus index : 378


looking for : Allocab index : 379


looking for : Ecotransite index : 380


looking for : Paris Best Way index : 381


looking for : Transp Logistique Sce Sotransls SOC index : 38

...



looking for : Trans Auto 69 index : 725


looking for : Nord Sud Express 69 index : 726


looking for : Guidoum Mohammed index : 727


looking for : Atlas Drive index : 728


looking for : Dz Voyages index : 729


looking for : Lyon Univers index : 730


looking for : Rhone distribution index : 731


looking for : My express driver index : 732


looking for : Vélimotion index : 733


looking for : Publitour index : 734


looking for : SPALVIERI CORENTIN index : 735


looking for : Teffahi Courses index : 736


looking for : International Freight Logistics index : 737


looking for : Vélimotion index : 738


looking for : Rh trans logistics index : 739


looking for : Jhon Philippe index : 740


looking for : SOCIETE BECYCLE index : 741


looking for : 3t index : 742


looking for : Lp Express SARL index : 743


looking for : Chetioui Hugo index : 744


looking for : Madani Mohamed index : 745


looking for : Lyon Continental Freight index : 746


looking for : Trans Eurocourses index



looking for : Chapelle Thierry index : 913


looking for : JetAirFly index : 914


looking for : TRANSNATURE index : 915


looking for : Murard Marc index : 916


looking for : ACTIV COLIS index : 917


looking for : Jarosson Thomas index : 918


looking for : Ndjibu Trésor index : 919


looking for : Diaby Nafie index : 920


looking for : Pichon Michel index : 921


looking for : Amolomalo Nathan Magnus index : 922


looking for : Amolomalo Nathan Magnus index : 923


looking for : Mekdad Abdelhamid index : 924


looking for : Clement Gilles index : 925


looking for : Berkani Karim index : 926


looking for : Fast Pack index : 927


looking for : BH TRANSERVICES index : 928


looking for : Samate Sadio index : 929


looking for : Davoine Marcel index : 930


looking for : Sandi Laiddine index : 931


looking for : URBAN LOGISTICS SERVICES index : 932


looking for : DRAME SOIREBA index : 933


looking for : KAKO TRANSPORTS index : 934


looking for : SNCF Immobilier DTI Sud Est in


looking for : Rahali Lamine index : 1095


looking for : Taxiway index : 1096


looking for : Hadjadj index : 1097


looking for : De Villard Régis index : 1098


looking for : De Villard Régis index : 1099


looking for : Taxi Daniel index : 1100


looking for : TRANSPORT EXPRESS LYONNAIS index : 1101


looking for : Alphadis index : 1102


looking for : Bousdira Adnane index : 1103


looking for : Autocar-Location.com index : 1104


looking for : Trans World Logistics index : 1105


looking for : Alameddine Véronique index : 1106


looking for : Philips Jean-Marc index : 1107


looking for : Siergie Romain index : 1108


looking for : Couquet Christophe index : 1109


looking for : Bj Conseil index : 1110


looking for : Damnati Adib Rédouane index : 1111


looking for : Mc Catty Christina index : 1112


looking for : MKSB EXPRESS index : 1113


looking for : Remli Fatima index : 1114


looking for : Benasna Ghali index : 1115


looking for : Barbach Zoubida index : 1116


looking f



looking for :  1er Choix Distribution  index : 1268


looking for :  2r Transports  index : 1269


looking for :  2sid  Tech  index : 1270


looking for :  2w Logistics  index : 1271


looking for :  3D Trans  index : 1272


looking for :  3N Tours  index : 1273


looking for :  3S Transit  index : 1274


looking for :  4 L Logistique Et Transport  index : 1275


looking for :  4 Line  index : 1276


looking for :  4 Rent - Transport  index : 1277


looking for :  4tl Med Transit  index : 1278


looking for :  4X4 Evasion  index : 1279


looking for :  A Five International Transport (A Five)  index : 1280


looking for :  A Plus Transport  index : 1281


looking for :  A Right Trans  index : 1282


looking for :  A Toute Vitesse Courses (Atvc)  index : 1283


looking for :  A&J Transport et Logistique  index : 1284


looking for :  A-j Transport  index : 1285


looking for :  A-med Transport & Logistique  index : 1286


looking for :  A-Trans  index : 1287


looking for :  A. Trans  



looking for :  Alam Transit Ammor (A.t.a.)  index : 1431


looking for :  Alaoui Transport  index : 1432


looking for :  Albatross Tours  index : 1433


looking for :  Alcus transport  index : 1434


looking for :  Alfa Evasion  index : 1435


looking for :  Alfa Tir  index : 1436


looking for :  AlHamd Logistique  index : 1437


looking for :  Alif Trans International  index : 1438


looking for :  Alina Transport  index : 1439


looking for :  Alintissar AlMouttahida  index : 1440


looking for :  Alisatransit  index : 1441


looking for :  Alitalia - Compagnia Aerea Italiana (Alitalia)  index : 1442


looking for :  Alitalia -Compagnia Aerea Italiana (Alitalia)  index : 1443


looking for :  Alitihad Négoce & Transport  index : 1444


looking for :  Alliance Logistics  index : 1445


looking for :  Alliance Shipping  index : 1446


looking for :  Alliance Transport  index : 1447


looking for :  Alliance Voyages  index : 1448


looking for :  Allied Movers  index : 1449


lookin

In [42]:
listCode

[0.5167518180362944,
 0.17874195011096014,
 0.5512065344640381,
 0.40036665638234215,
 0.20081752355864096,
 0.20081752355864096,
 0.239072830937752,
 0.273442717843107,
 0.16210354727051451,
 0.20081752355864096,
 0.0,
 0.44489059420152655,
 0.13956908313231214,
 0.26951761324603224,
 0.37570207694760915,
 0.26951761324603224,
 0.0,
 0.239072830937752,
 0.16210354727051451,
 0.44489059420152655,
 0.43750847963528,
 0.15055696960204948,
 0.15055696960204948,
 0.15055696960204948,
 0.16171378066252898,
 0.2446216892517935,
 0.2060836350139382,
 0.24552468987762036,
 0.16171378066252898,
 0.1908740661302035,
 0.12725898700975247,
 0.15055696960204948,
 0.15055696960204948,
 0.2521387069452626,
 0.16171378066252898,
 0.15055696960204948,
 0.16171378066252898,
 0.1337850929463124,
 0.37754665716077884,
 0.4255389501816191,
 0.24643776495971234,
 0.2842874150525866,
 0.16171378066252898,
 0.3933038316562642,
 0.2941480135010239,
 0.3478109916399369,
 0.2842874150525866,
 0.18306901354215366

In [43]:
dfCatClass['Nom'] = listNom
dfCatClass['Code Secteur'] = listCode
dfCatClass['Nom Secteur'] = listNomSecteur

In [44]:
dfCatClass

,Nom,Code Secteur,Nom Secteur
0,Allo Fret,0.516752,TRANSPORT ROUTIER INTERNATIONAL
1,DHL Express,0.178742,CONSEIL (EN LOGISTIQUE) - FORMATION ET EMPLOI
2,AAC Globe Express,0.551207,TRANSPORT ROUTIER INTERNATIONAL
3,France Express,0.400367,TRANSPORT EXPRESS OU URGENT
4,DHL Express,0.200818,TRANSPORTS EN CITERNE
...,...,...,...
1584,Awell Ttl,0.161714,LOGISTIQUE AVANCÉE
1585,Axalys Logistics Maroc,0.150557,TRANSPORTS SPÉCIALISÉS
1586,Aya Intercontinental,0.410363,TRANSPORT INTERCONTINENTAL
1587,Ayan Transit,0.161714,LOGISTIQUE AVANCÉE


In [45]:
dfCatClass.rename(columns = {"Code Secteur" : "Similarity"}, inplace = True)

In [46]:
listCode = []
listSecteur = []
for i in range(1589):
    for j in range(37):
        if  dfCatClass['Nom Secteur'][i] == dfCategories['Nom Secteur'][j]:
            listCode.append(dfCategories['Code Secteur'][j])
            listSecteur.append(dfCategories['Nom Secteur'][j])
            break

In [47]:
dfCatClass['Code Secteur'] = listCode

In [48]:
dfCatClass

,Nom,Similarity,Nom Secteur,Code Secteur
0,Allo Fret,0.516752,TRANSPORT ROUTIER INTERNATIONAL,1.1.7
1,DHL Express,0.178742,CONSEIL (EN LOGISTIQUE) - FORMATION ET EMPLOI,1.5.2
2,AAC Globe Express,0.551207,TRANSPORT ROUTIER INTERNATIONAL,1.1.7
3,France Express,0.400367,TRANSPORT EXPRESS OU URGENT,1.1.5
4,DHL Express,0.200818,TRANSPORTS EN CITERNE,1.1.3
...,...,...,...,...
1584,Awell Ttl,0.161714,LOGISTIQUE AVANCÉE,1.2.3
1585,Axalys Logistics Maroc,0.150557,TRANSPORTS SPÉCIALISÉS,1.1.2
1586,Aya Intercontinental,0.410363,TRANSPORT INTERCONTINENTAL,1.1.8
1587,Ayan Transit,0.161714,LOGISTIQUE AVANCÉE,1.2.3


# Level 3 : Maximum similarity between catg and sub catg

In [49]:
dfCatClass.rename(columns = {"Code Secteur" : "Categorie code"}, inplace = True)
dfCatClass.rename(columns = {"Similarity" : "Categorie Similarity"}, inplace = True)
dfCatClass.rename(columns = {"Nom Secteur" : "Categorie"}, inplace = True)

In [51]:
dfCatClass.head()

,Nom,Categorie Similarity,Categorie,Categorie code
0,Allo Fret,0.516752,TRANSPORT ROUTIER INTERNATIONAL,1.1.7
1,DHL Express,0.178742,CONSEIL (EN LOGISTIQUE) - FORMATION ET EMPLOI,1.5.2
2,AAC Globe Express,0.551207,TRANSPORT ROUTIER INTERNATIONAL,1.1.7
3,France Express,0.400367,TRANSPORT EXPRESS OU URGENT,1.1.5
4,DHL Express,0.200818,TRANSPORTS EN CITERNE,1.1.3


In [53]:
dfCatClass['Sub Categorie Similarity'] = dfSubCatClass['Similarity']
dfCatClass['Sub Categorie'] = dfSubCatClass['Nom Secteur']
dfCatClass['Sub Categorie code'] = dfSubCatClass['Code Secteur']

In [54]:
dfCatClass.head()

,Nom,Categorie Similarity,Categorie,Categorie code,Sub Categorie Similarity,Sub Categorie,Sub Categorie code
0,Allo Fret,0.516752,TRANSPORT ROUTIER INTERNATIONAL,1.1.7,0.563834,- Transport de meubles,1122
1,DHL Express,0.178742,CONSEIL (EN LOGISTIQUE) - FORMATION ET EMPLOI,1.5.2,0.235836,- Transport et livraison,1231
2,AAC Globe Express,0.551207,TRANSPORT ROUTIER INTERNATIONAL,1.1.7,0.618945,- Transport de meubles,1122
3,France Express,0.400367,TRANSPORT EXPRESS OU URGENT,1.1.5,0.499754,- Transport express national,1151
4,DHL Express,0.200818,TRANSPORTS EN CITERNE,1.1.3,0.283636,- Transport et livraison,1231


In [55]:
writer = pd.ExcelWriter('Total DataClassification .xlsx')
dfCatClass.to_excel(writer)
writer.save()

In [60]:
dfFinalClassif = DataFrame(columns = ['Nom','Secteur','Code Secteur'])

In [66]:
# Take categorie and code categorie from the classification with max similarity

listSecteur = []
listCodeSecteur = []

for i in range(1589):
    maxSimilarity = max(dfCatClass['Categorie Similarity'][i], dfCatClass['Sub Categorie Similarity'][i])
    if maxSimilarity == dfCatClass['Categorie Similarity'][i]:
        listSecteur.append(dfCatClass['Categorie'][i])
        listCodeSecteur.append(dfCatClass['Categorie code'][i])
    elif maxSimilarity == dfCatClass['Sub Categorie Similarity'][i]:
        listSecteur.append(dfCatClass['Sub Categorie'][i])
        listCodeSecteur.append(dfCatClass['Sub Categorie code'][i])
    else:
        listSecteur.append("Prestations transport et logistique")
        listCodeSecteur.append("1")

In [74]:
for i in range(1589):
    if listSecteur[i].find('-') != -1:
        listSecteur[i] = listSecteur[i][1:]
        

In [76]:
dfFinalClassif['Nom'] = dfCatClass['Nom']
dfFinalClassif['Secteur'] = listSecteur
dfFinalClassif['Code Secteur'] = listCodeSecteur

In [78]:
writer = pd.ExcelWriter('Data Classification.xlsx')
dfFinalClassif.to_excel(writer)
writer.save()